# DAGs - a generalization of the alignment problem
##Specification
- Name: problem15 in Rosalind
- Name your notebook as: DAG.ipynb
- options: none
- input: filename passed as first parameter to main
- output: a text file. ( using print ( .... file=someFileObject) is a handy way to do this after you have opened someFileObject as a text file). I find it handy to name these files by creating a string by concatenating the string named infile with ".out" ... rosalind5.txt.out ( for example).
-Rosalind Problem Name: Find the longest path in a DAG

As always, include a Inspection Intro Markdown that describes your specific algorithm at the beginning of the notebook, and another Inspection Results markdown at the end of the notebook that documents: your inspection team, the findings of the team, and your resolution of those findings.

Please submit your notebook, an example of one of the Rosalind files that you ran and passed, and the output that your program generated as a text file.

## Description
As a prelude to an upcoming assignment in profile alignments, please consider the related problem of generalized weighted-DAGs (directed acyclic graph). Rosalind has a very nice example that includes weighted edges connecting nodes. As applied to a simple pairwise alignment, nodes represent a specific assignment of a character pairing between a character in one of the sequences to either an initial insert/delete (indel), an extension of that indel, or a match/mismatch. 

Unlike classical pairwise alignment, the specific order to consider the resolution of scores in the graph is established using the topology of the graph ( topological ordering). This ordering will avoid recursion in developing the final score. This is another way of using Dynamic programming in a more generalized way. ( pp247-52 in Compeau and the detour on pp287-88).

## Hints
1) scoring of nodes that have no inputs (or possibly incomplete inputs) must be done carefully. Choose these wisely. In a pairwise alignment, these nodes would be considered boundary nodes.

2) The problem is quite clear on the starting/ending node to consider. Make sure to evaluate these properly.

3) each node can have many inputs or none. In the generalized DAG, there is no limit on the number of inputs. Consider a data structure here that allows you to represent the incoming edge along with a score that is computed by adding the weight of that incoming edge to the score of its preceding node.

4) As you write these classes, consider that these may be usable in the future.

5) sets and dictionaries require keys that are immutable. Those keys can be integers, or floats, or tuples. Think carefully about using tuples as containers-that-can-be-keys.

6) python has a container called a namedtuple. These are available in the container module. They are quite handy.

7) python has two functions called any() and all(); any() evaluates to true if any of its iterable arguments are True; all() evaluates to True if all of its iterable arguments are true. None, for example, evaluates to False. 

8) consider implementing a helper method called _argMax_. If you happen to have a dictionary of values, it returns the key whose paired-value is maximal among all values in the dictionary.

9) Carefully consider the implementations that are provided in the text. Specifically, the topology ordering pseudocode removes edges from the constructed DAG. This is a bit annoying, considering that we need that DAG for scoring and backtracking. What is the purpose that removing an edge is attempting to perform? Look over the Wikipedia article on "Dataflow" for an idea as to how you might avoid destroying your graph.

10) My implementation has specific objects that are nodes, edges, and the DAG itself. It is implemented in about 120 lines including whitespace.

11) Rosalind will provide test cases where the proscribed beginning and ending node have inputs/outputs that may induce cycles. In these problems, remember that we are considering the DAG that is described between (inclusively) the start and end node so the inputs to start don't matter; the outputs from end don't matter either.

 

Here is a template to use.

## Inspection Intro

In [ ]:
class Graph:
    def __init__(self):
        self.graph = {}

    def getNodes(self):
        '''get all the nodes of the graph'''
        return list(self.graph.keys())

    def getEdges(self):
        '''get all the edges of the graph in from of (node1, node2)'''
        edges = []
        for vertex in self.getNodes():
            for node in self.graph[vertex]:
                edges.append((vertex, node))

    def addNodes(self, node):
        '''add a new node to the graph'''
        if node not in self.getNodes():
            self.graph[node] = []

    def addEdges(self, n1, n2):
        '''add new edge to the graph'''
        if n1 not in self.getNodes():
            self.addNodes(n1)
        if n2 not in self.getNodes():
            self.addNodes(n2)
        if n2 not in self.graph[n1]:
            self.graph[n1].append(n2)

    def add_weighted_edge(self, n1, n2, w):
        '''add new edge to the graph with wight'''
        if n1 not in self.getNodes():
            self.addNodes(n1)
        if n2 not in self.getNodes():
            self.addNodes(n2)
        if n2 not in self.graph[n1]:
            self.graph[n1].append((n2, w))
            
    def get_weight(self, n1, n2):
        '''get weight of node from n1 to n2 on a
        weighted graph'''
        for edge in self.graph[n1]:
            if edge[0] == n2:
                return edge[1]

    def getSuccessors(self, node):
        '''get all the successors of certain node on graph'''
        return list(self.graph[node])

    def getPredecessors(self, node):
        '''get all the predecessor of certain node on graph'''
        pred = []
        for k in self.getNodes():
            if node in self.graph[k]:
                pred.append(node)
        return pred

    def inDegree(self, node):
        '''get number of in edges of certain node'''
        return len(self.getPredecessors(node))


In [ ]:
import math
import copy
import random

class DAG(Graph):
    def __init__(self, begin, end):
        Graph.__init__(self)
        self.begin = begin
        self.end = end

    def topological_order(self, graph):
        '''Find a topological ordering of a directed acyclic graph 
        input:
            The adjacency list of a graph
        output:
            a topological ordering of this graph
        '''
        list = []
        # initialize candidates
        candidates = []
        for node in graph.getNodes():
            if graph.inDegree(node) == 0:
                candidates.append(node)

        while len(candidates) != 0:
            # select arbitrary node from candidates
            a = random.choice(candidates)
            list.append(a)
            candidates.remove(a)

            # find outgoing edge from a to another node b
            successors = graph.getSuccessors(a)
            for b in successors:
                # remove edge (a, b) from graph
                for n2 in graph.graph[a]:
                    if n2 == b:
                        graph.graph[a].remove(b)
                if graph.inDegree(b) == 0:
                    candidates.append(b)

        # check wether the graph is DAG or not
        for node in graph.getNodes():
            if len(graph.graph[node]) != 0:
                return "the input Graph is not a DAG"

        return list
    
    def backtrack(self, previous):
        '''find the optimal path from end to start
        input:
            a dictionary storing previous choice
        output:
            list of path from start to end
        '''
        path = [self.end]
        cur = path[0]
        while cur != self.begin: 
            next = previous[cur]
            path.insert(0, next)
            cur = next
        return path
            
    
    def get_no_pred_list(self, topological_list, graph):
        '''check where predecessor is legitimate
        input:
            topological list
            unweighted graph 
        output:
            list of node with no pred
        '''
        noPred = []
        for i in range(1, len(topological_list)):
            pred = []
            for k in graph.keys():
                if topological_list[i] in set(graph[k]):
                    pred.append(k)
            # handle when node has no predecessor
            if len(pred) == 0:
                noPred.append(topological_list[i])
                continue
        return noPred

    def longest_path(self, w_graph, graph):
        '''Find the longest path and the highest score in a DAG
        input:
            weighted graph object
            unweighted graph object
        output
            tuple(highest score, list of path)
        '''
        # create dictionary to store Score
        sd = Graph()
        for node in graph.getNodes():
            sd.graph[node] = -math.inf
        
        # get topological order
        g2 = copy.deepcopy(graph.graph) # intact
        
        while True:
            list = self.topological_order(graph)
            if list[0] == self.begin and list[-1] == self.end:
                break

        # set source node score
        sd.graph[list[0]] = 0
        # call get no pred list
        noPred = self.get_no_pred_list(list, g2)
        # find score
        previous = {} # record previous choice
        for i in range(1, len(list)):
            # get predecessors
            pred = []
            for k in g2.keys():
                if list[i] in set(g2[k]):
                    pred.append(k)
            # handel when node has no predecessor
            if list[i] in noPred:
                continue
            # handel when node has predecessors that dose not have predecessor
            check = all(item in noPred for item in pred)
            if check == True:
                continue    
            temp = [] # store sb + weight of edge from b to a  
            for p in pred:
                weight = int(w_graph.get_weight(p, list[i]))
                value = sd.graph[p] + weight
                temp.append(value)
            max_weight = max(temp)
            ind = temp.index(max_weight)
            previous[list[i]] = pred[ind] # node choose its optimal previous path 
            # update score dict
            sd.graph[list[i]] = max_weight
        score = sd.graph[list[-1]]

        # backtracking 
        path = self.backtrack(previous)
        
        return score, path

def main(inFile=None):
    '''
    Do the main thing
    '''
    wg = Graph()
    tg = Graph()  # unweighted graph to topological sort
    with open(inFile, 'r') as fh:
        begin = fh.readline().rstrip()
        end = fh.readline().rstrip()
        thisDAG = DAG(begin, end)
        # read file
        for line in fh:
            n1 = line.strip().split('->')[0]
            n2 = line.strip().split('->')[1].split(':')[0]
            tg.addEdges(n1, n2)
            weight = line.strip().split('->')[1].split(':')[1]
            wg.add_weighted_edge(n1, n2, weight)
    # write into file
    res = thisDAG.longest_path(wg, tg)
    fw = open('DAGresult.txt', 'w')
    fw.write(str(res[0]) + '\n')
    result = ''
    for i in res[1]:
        result = result + str(i) + "->"
    result = result[0:len(result)-2]
    fw.write(result)

if __name__ == "__main__":
    main(inFile='rosalind_ba5d.txt')


## Inspection Results

### 1. William Gao:

- great comments and docstrings. I also like that you are using a base `Graph` class and inheriting it for your DAG
- I'm not entirely sure why you are recomputing the topological order on lines 100 - 103, maybe add some comments there?

That's also something I was not sure. Beacuse it seems like sometimes there are multiple topological order exists if I didn't choose the self.begin and self.end. Maybe I should do that inside the topological sort method.

### 2. Aleksis Korhonen:

- Looks great! Code is clear and easy to read. Having a second unweighted graph to do the topological ordering is a very interesting solution. I'm a fan of the on demand node creation when adding edges - I went with this strategy as well.
- I think I understand the use of the deepcopy here: in order to preserve the state of the original graph.


### 3. Lucy Zheng
- I like your predecessors function along with the in degree function that shows the number of incoming edges
- The doc strings and comments are very detailed about your code, overall your code is very neat


